In [ ]:
import torch 
import numpy as np
import matplotlib.pyplot as plt
import importlib
import sys

In [ ]:
seed = 70
model_weights = '/home/kuba/projects/puff/paper00/experiments/old/04session-split00/runs/same_config_run/3/best_dev_model.pt'
model_def = '/home/kuba/projects/puff/paper00/unet.py'
data_path = '/home/kuba/projects/puff/paper00/experiments/old/04session-split00/data/test.pt'

In [ ]:
X, y = torch.load(data_path, weights_only=True)

In [ ]:
X[0].shape

In [ ]:
def viz(i, data, to_transpose=True):
    to_plot = torch.hstack((data[i], data[i+1]))
    print(to_plot.shape)
    if to_transpose:
        plt.plot(to_plot.T);
    else:
        plt.plot(to_plot);
    plt.vlines(x=1024, ymin=-10, ymax=10, linestyles='--', colors='black', alpha=0.18)
    plt.show();

In [ ]:
len(X)

In [ ]:
#now can we un-shuffle this
np.random.seed(seed)
perm_idx = np.random.permutation(len(X))
reverse =  np.empty_like(perm_idx)
reverse[perm_idx] = np.arange(len(perm_idx))

In [ ]:
X_fixed = X[reverse]
y_fixed = y[reverse]

In [ ]:
X[:,:3].shape

In [ ]:
i = 91
viz(i, X[:,:3])
viz(i, X_fixed[:,:3])

In [ ]:
sum(y_fixed[91] == 1)

In [ ]:
i = 91
viz(i, y, to_transpose=False)
viz(i, y_fixed, to_transpose=False)

In [ ]:
torch.where(y_fixed.sum(axis=1) > 0)

In [ ]:
# Add  the model def to the path to import the class
module_path, file_name = model_def.rsplit('/', 1)
file_prefix = file_name.split('.')[0]
module_path, file_prefix

In [ ]:
sys.path.append(module_path)
module = importlib.import_module(file_prefix)
model_class = getattr(module, 'Model')

#add an needed specifications
model_obj = model_class()

In [ ]:
model_obj.load_state_dict(torch.load(model_weights, weights_only=True))

In [ ]:
model_obj.eval()
with torch.no_grad():
    output = model_obj(X_fixed[90:100])

In [ ]:
plt.figure(figsize=(18,10))
plt.plot(torch.sigmoid(output.flatten()));
plt.plot(y_fixed[90:100].flatten());

In [ ]:
thresh_pred = (torch.sigmoid(output.flatten()) > 0.8).long()

In [ ]:
from scipy.ndimage import binary_closing

# Apply after sigmoid and thresholding
closed_thresh_pred = binary_closing(thresh_pred, structure=np.ones(100)) - 0.004


In [ ]:
plt.figure(figsize=(18,10))
plt.plot(closed_thresh_pred);
plt.plot(y_fixed[90:100].flatten());